# ![Project Overview](images/readme/Project%20Overview.png)
This project aims to build a convolutional neural network that can classify images of coffee beans into one of four categories representing various stages of the roasting process: raw, light, medium, and dark. The model will take jpeg images of coffee beans as an input, analyze the image using convolutional layers, and make a prediction about the roast stage.

This project is important to pursue because it demonstrates how artificial intelligence (AI) can be used to improve processes and outcomes in the coffee industry. By leveraging the power of machine learning, coffee roasters can achieve greater consistency in their roasts, which can lead to higher quality coffee and increased customer satisfaction. Additionally, this project is just the tip of the iceberg when it comes to the potential for innovation in the coffee industry utilizing AI technology. There are many other applications of machine learning in the coffee industry, such as predicting consumer preferences, optimizing supply chain logistics, and improving the efficiency of coffee farming. As such, this project represents an exciting opportunity to explore the possibilities of AI in the coffee industry and beyond.  

The project can be broken down into the following steps:  
- Collect a dataset of labeled images of coffee beans in the four roast level categories.    
- Preprocess the images (resize, crop, normalize, etc.) to make them suitable for training the model.  
- Iterate over various versions of network architecture, hyperparameters and training protocols.   
- Evaluate the performance of each model version on a test dataset to determine accuracy and other key performance indicators (KPI)  
- Deploy the model to a system that can accept jpeg images as input, classify them, and alert workers when the desired roast is achieved. 

# ![Business Use Case](images/readme/Business%20Use%20Case.png)

This project has the potential to significantly improve the coffee roasting process in terms of quality control, labor efficiency, and profitability. By using a camera positioned above the roasting bin to periodically capture images and analyze the roast level of the beans, the system can help coffee roasters achieve greater consistency in their roasts. This can lead to higher quality coffee and increased customer satisfaction.  

In addition, this system can help reduce waste by alerting workers when the beans are overcooked or undercooked, thereby minimizing the amount of coffee that must be discarded. This reduction in waste can have a significant impact on a roaster's bottom line, as it reduces the cost of materials and increases profitability.  

Furthermore, this system can help workers be more productive by allowing them to focus on other tasks while the roasting process is underway. Since the system can monitor the roasting process and alert workers when the desired roast level is achieved, workers do not need to pay as close attention to each roasting bin. This can free up time and resources that can be directed towards other areas of the business, further increasing efficiency and profitability.
Overall, this project represents a powerful tool for coffee roasters looking to improve their operations and achieve greater success in the highly competitive coffee industry. By utilizing AI technology to improve quality control, reduce waste, and increase labor efficiency, coffee roasters can deliver a better product to their customers and increase their bottom line.

# ![Data Understanding](images/readme/Data%20Understanding.png)  
The dataset used in this project comes from [kaggle.com](https://www.kaggle.com/datasets/gpiosenka/coffee-bean-dataset-resized-224-x-224) and consists of 1200 training images and 400 test images. Each sample is equally distributed between the four classes, with 300 images per class in the training set and 100 images per class in the test set. All the images are of a single coffee bean and are of size 224x224 pixels. The dataset's class labels are (Dark, 0), (Green, 1), (Light, 2), and (Medium, 3). The images were captured using an iPhone 12 mini, which means that the images are of high quality and consistent.
To further test the model and the dataset's limitations, I will also create a small validation dataset using our own iPhone 13 pro camera. This dataset will include images of both single coffee beans, like the training and test data, as well as images of multiple beans. By doing so, I hope to evaluate the model's performance in detecting the stage of the roasting process of both individual and multiple beans.
Overall, I believe that this dataset provides a good representation of the different stages of coffee bean roasting, and I am confident that our model can successfully classify coffee bean images.


## Importing Libraries and Other Requirements
This projects utilizes a battery of industry standard first and third part libraries. With one exception being the Project Toolkit module with the alias 'ptk', this module is written custom to visualized and gether performance statistics of the types of models being built within this notebook. I have endeavared to provide exhaustive comments, doc-strings, and other pertinent documentation in the source code and throughout this repo to make the project toolkit as efficient and easy to use as possible.

In [ ]:
# cloneing source repo to access necessary Python files
# replace with your own forks URL if necessary
! git clone https://github.com/Zeth-Abney/CoffeeBeanClassifier.git

In [ ]:
# dependencies for data preprocessing
import os
import numpy as np
import pandas as pd
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array

# dependencies for model compilation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

from keras import layers
from keras import models
from keras import optimizers
from keras.metrics import Recall
from keras.regularizers import l2

from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.models import load_model

# dependencies for model diagnostics
from sklearn.metrics import confusion_matrix, classification_report
from CoffeeBeanClassifier import project_toolkit as ptk

import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline


## Getting the Data
To make it easy for others to reproduce this project elsewhere, I highly recommend using [Google Colab](https://colab.research.google.com/) to run this notebook. Colab eliminates the need for local computing and storage resources, so you won't have to worry about any environment or dependency issues.

To get started, first fork the [original repository](https://github.com/Zeth-Abney/CoffeeBeanClassifier) and input the URL of your own fork under the GitHub tab of the Colab splash page. Once you have the notebook open in Google Colab, run the rest of the cells in this sub-section.

In order to download the data you must access the Kaggle API, you'll need to input your own Kaggle username and API key as values to the variables in the first cell on lines 9 and 10, respectively.

In [ ]:
# handling the api key authentication
# .kaggle folder to hold api key
!mkdir /root/.kaggle

# creating the kaggle.json file to hold the username and key
!touch /root/.kaggle/kaggle.json

# credentials for the api authentication
user = '' # add your kaggle username here
key = '' # add your kaggle API key here
api_token = {'username':user, 'key':key}

# saving the credentials in a file
import json
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

# changing permissions for the file to be read
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# using kaggle api to acquire data
! kaggle datasets download gpiosenka/coffee-bean-dataset-resized-224-x-224

In [ ]:
# unzipping kaggle download into 'data' folder
import zipfile
with zipfile.ZipFile('/content/coffee-bean-dataset-resized-224-x-224.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')

In [ ]:
# listing filepaths for renaming
data_sample_list = os.listdir('data')[:2]

root = 'data/'
rename_dict = {
    'Green':'A. Green',
    'Light':'B. Light',
    'Medium':'C. Medium',
    'Dark': 'D. Dark'
}

# renaming filepaths at the class level so clas labels are ordinal
for sample in data_sample_list:
  class_list = os.listdir(root+sample)
  for data_class in class_list:
    old_name = root+sample+'/'+data_class
    new_name = root+sample+'/'+rename_dict[data_class]
    os.rename(old_name,new_name)

In [ ]:
# file paths to data directories
train_dir = "data/train/"
test_dir = "data/test/"

### Creating Your Own Validation Data
Utilizing the preprocess_new_image() function from the project toolkit you can create your own validation dataset using images of coffee beans take on your very own phone or camera. They will need to be cropped with a 1:1 aspect ratio and the afformentioned function will take care of the rest of the preprocessing. 

First upload your images to the folder 'preprocessing (val)', and then run the two cells below. 

Later in the 'Validation Testing' section the images in the 'val' directories will be used the create another ImageDataGenerator object the can be tested and visualized using other function from the custom project_toolkit.py module.

In [ ]:
# creating directories for validation data preprocessing
os.mkdir('data/preprocessing (val)')
os.mkdir('data/val')

In [ ]:
# read and preprocess new images from source to target directories
ptk.preprocess_new_image('data/preprocessing (val)','data/val')

# Summary Dataset Visualization

In [ ]:

# dictionary of volume of data per class per sample
# len(os.listdir(train_dir+"Dark"))
class_balance_dict = {
    "Train Sample":[len(os.listdir(train_dir+"A. Green")),
                    len(os.listdir(train_dir+"B. Light")),
                    len(os.listdir(train_dir+"C. Medium")),
                    len(os.listdir(train_dir+"D. Dark"))],
    "Test Sample":[len(os.listdir(test_dir+"A. Green")),
                   len(os.listdir(test_dir+"B. Light")),
                   len(os.listdir(test_dir+"C. Medium")),
                   len(os.listdir(test_dir+"D. Dark"))]
                    }

# # class balance dictionary as a dataframe with row index representing class and column index representing sample
class_balance_df = pd.DataFrame(class_balance_dict,index=['Green','Light','Medium','Dark'])
class_balance_df

In [ ]:
# Create colors
cmap = plt.cm.copper
color_indexes = [cmap(1.0), cmap(0.75), cmap(0.5), cmap(0.25)]

# create stacked bar chart for students DataFrame
class_balance_df.T.plot(kind='bar', stacked=True, color=color_indexes)

# Add Title and Labels
plt.title('Class Balance by Sample')
plt.xlabel('Sample Split')
plt.ylabel('Percent Class Balance')

plt.legend(bbox_to_anchor=(1, 1))

plt.show()

In [ ]:
# Define the figure and subplot layout
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
pie_colors = [cmap(1.0), cmap(0.75), cmap(0.5), cmap(0.25)]

# Set the radius of the first pie chart
radius1 = 1

# Create the first pie chart
wedges1, texts1 = ax1.pie(class_balance_df['Train Sample'], radius=radius1,colors=pie_colors)

# Set the radius of the second pie chart
radius2 = radius1 * 2 / 3

# Create the second pie chart
wedges2, texts2 = ax2.pie(class_balance_df['Test Sample'], radius=radius2,colors=pie_colors)

# Add a legend to the second pie chart
ax2.legend(wedges2, ['Green','Light','Medium','Dark'], loc="right")

# Set the title for each subplot
ax1.set_title("Train Sample (1200 data points)")
ax2.set_title("Test Sample (400 data points)")

# Show the plot
plt.show()

# ![Data Preparation](images/readme/Data%20Preparation.png)
To prepare the image data for modeling, I utilized the ImageDataGenerator class from Keras to read in the JPEG files and directly access the image data as matrices. This method makes it easy to load in and preprocess large datasets. The ImageDataGenerator objects were set to normalize the pixel values to a scale of 0-1 using the rescale argument. I also set the color_mode argument to grayscale since I only need to work with a single channel for these images.

Normalizing the pixel values and setting to grayscale are standard practices for image classification tasks. Normalizing the pixel values scales down the pixel values to a range that is better suited for machine learning algorithms, while setting the images to grayscale removes color as a variable in the analysis, simplifying the model's learning process.

In [ ]:
# instatiating a data degenerater for each split sample 
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# 25 is a common factor or both 1200 and 400
batch_size = 25

In [ ]:
train_data_generator = train_datagen.flow_from_directory(
                       train_dir,
                       target_size=(224,224),
                       batch_size= batch_size,
                       class_mode='categorical',
                       color_mode='grayscale')

test_data_generator = test_datagen.flow_from_directory(
                      test_dir,
                      target_size=(224,224),
                      batch_size= batch_size,
                      class_mode='categorical',
                      color_mode='grayscale')

# ![Modeling Methods](images/readme/Modeling%20Methods.png)  
For this project, I developed a convolutional neural network (CNN) using Keras to classify different levels of roast for coffee beans. A CNN is an ideal model choice for image classification tasks like this one because it can effectively extract important features from the input images through convolutional layers, and can then use pooling layers to reduce dimensionality and improve computational efficiency. Additionally, CNNs are able to learn hierarchical representations of the input images, which is especially useful for image recognition tasks where features like edges, shapes, and textures can be learned at lower layers and combined to identify more complex features like object shapes and patterns at higher layers.

Keras is a great tool for building CNNs because it provides a user-friendly and high-level interface for constructing and training deep learning models. Its simple and intuitive syntax allows for rapid prototyping and iteration, making it ideal for experimentation and development. Keras also offers a variety of pre-built neural network layers and optimization algorithms, which can save time and reduce the complexity of building deep learning models from scratch. Overall, using Keras made it easier for me to focus on the specific details of my project rather than worrying about low-level implementation details.

## Baseline Model

The baseline model consists of three convolutional layers separated by three max pooling layers, a flattening layer, and then two Dense layers and then another Dense output layer. The model is compiled using the Adam gradient descent optimizer and its loss is measured based on categorical cross entropy. The filter count and kernal size for each layer has been optimized for performance as well as training speed, and are set in a way that the filters' areas of observation do not overlap. The model is trained over a single epoch in 48 training steps and 16 validation steps in batches of 25, so each data point, both training and test, are observed only once throughout training.  

The base model has an overall accuracy of about 75%, and it appears to be more dependable the darker the bean is. The loss is in a desirable range, so I hesitate to say there is overfitting or a strong bias at play here, I believe its more likely that a lack of image contrast in the lighter beans provides less information for the model to digest. 


In [ ]:
# Building the model
base_model = Sequential()

# 3 convolutional layers
base_model.add(Conv2D(56, (4,4), activation='relu', input_shape = (224,224,1)))
base_model.add(MaxPooling2D(pool_size=(2,2)))

base_model.add(Conv2D(112, (2,2), activation='relu'))
base_model.add(MaxPooling2D(pool_size=(2,2)))

base_model.add(Conv2D(112, (2,2), activation='relu'))
base_model.add(MaxPooling2D(pool_size=(2,2)))

base_model.add(Flatten())

base_model.add(Dense(56,activation='relu'))
base_model.add(Dense(56, activation='relu'))

base_model.add(Dense(4))
base_model.add(Activation("softmax"))

# Compiling the base_model using some basic parameters
base_model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
base_model_results = base_model.fit(
                     train_data_generator,
                     batch_size=batch_size,
                     epochs=1,
                     steps_per_epoch=48,
                     validation_data=test_data_generator,
                     validation_batch_size=batch_size,
                     validation_steps=16
                     )

In [ ]:
base_KPI, base_report, base_matrix, base_labels = ptk.evaluate_model(base_model,test_data_generator)

In [ ]:
print(base_report)

In [ ]:
ptk.viz_confusion_matrix(base_matrix,'Baseline Model')

In [ ]:
ptk.viz_class_balance_comparison(base_labels,"Baseline Model")

In [ ]:
ptk.viz_training_history(base_model_results,'Baseline Model')

## Optimizing the Fit Function

To improve the model's accuracy, I tested different ways to train the model. I first tried varying the number of epochs and batch size.I trained the model for 4, 8, and 12 epochs with steps_per_epoch such that every training image is only seen once. Then I tried 12 epochs again with enough steps that every training image is seen exactly twice. However, the improvement in accuracy was not satisfactory.

Next, I tried increasing the number of steps per epoch to see if the model's performance would improve. I set the number of steps per epoch to 48 so that the entire training dataset is seen for each epoch, and trained the model for 12 epochs. This resulted in a notable improvement in accuracy and loss.

Seeing that there was still room for improvement, I decided to increase the number of epochs to 100 with a step size of 48. To prevent overfitting, I added an EarlyStoppage callback set to monitor validation accuracy and halt training after 5 epochs of this metric not improving. The training went on for 16 epochs and this model performed nearly perfectly on the testing data, achieving an accuracy of .

Overall, this second model iteration showed that the model's performance could be significantly improved by adjusting the training parameters and adding an EarlyStoppage callback.


In [ ]:
# define early stopping callback
early_stop = EarlyStopping(monitor='val_accuracy', patience=5)

In [ ]:
# Building the model
fit_model = Sequential()

# 3 convolutional layers
fit_model.add(Conv2D(56, (4,4), activation='relu', input_shape = (224,224,1)))
fit_model.add(MaxPooling2D(pool_size=(2,2)))

fit_model.add(Conv2D(112, (2,2), activation='relu'))
fit_model.add(MaxPooling2D(pool_size=(2,2)))

fit_model.add(Conv2D(112, (2,2), activation='relu'))
fit_model.add(MaxPooling2D(pool_size=(2,2)))

fit_model.add(Flatten())

fit_model.add(Dense(56,activation='relu'))
fit_model.add(Dense(56, activation='relu'))

fit_model.add(Dense(4))
fit_model.add(Activation("softmax"))

# Compiling the fit_model using some basic parameters
fit_model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
fit_model_results = fit_model.fit(
                     train_data_generator,
                     batch_size=batch_size,
                     epochs=100,
                     steps_per_epoch=48,
                     validation_data=test_data_generator,
                     validation_batch_size=batch_size,
                     validation_steps=2,
                     callbacks=[early_stop]
                     )

In [ ]:
fit_KPI, fit_report, fit_matrix, fit_labels = ptk.evaluate_model(fit_model,test_data_generator)

In [ ]:
print(fit_report)

In [ ]:
ptk.viz_confusion_matrix(fit_matrix,"Sixteen Epochs,  Gross Observation")

In [ ]:
ptk.viz_class_balance_comparison(fit_labels, "Sixteen Epochs,  Gross Observation")

In [ ]:
ptk.viz_training_history(fit_model_results,"Sixteen Epochs,  Gross Observation")

## Network Regularization
For the third model iteration, I applied L2 regularization to the two Dense layers in the model. I tested different kernel weights, including 0.005 and 0.005, 0.0005 and 0.0005, and 0.00005 and 0.0005. The best performing model was the one with the last combination of kernel weights where the earlier layer has a kernal weight lighter than the later one. This model is nearly perfect, missclassifying only about two of the 400 testing data points so it is the final model I will proceed with for proof-of-concept and validation testing. 

In [ ]:
# Building the model
L2_model = Sequential()

# 3 convolutional layers
L2_model.add(Conv2D(56, (4,4), activation='relu', input_shape = (224,224,1)))
L2_model.add(MaxPooling2D(pool_size=(2,2)))

L2_model.add(Conv2D(112, (2,2), activation='relu'))
L2_model.add(MaxPooling2D(pool_size=(2,2)))

L2_model.add(Conv2D(112, (2,2), activation='relu'))
L2_model.add(MaxPooling2D(pool_size=(2,2)))

L2_model.add(Flatten())

L2_model.add(Dense(56,activation='relu',kernel_regularizer=l2(0.00005)))
L2_model.add(Dense(56, activation='relu',kernel_regularizer=l2(0.0005)))

L2_model.add(Dense(4))
L2_model.add(Activation("softmax"))

# Compiling the L2_model using some basic parameters
L2_model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
L2_model_results = L2_model.fit(
                     train_data_generator,
                     batch_size=batch_size,
                     epochs=15,
                     steps_per_epoch=48,
                     validation_data=test_data_generator,
                     validation_batch_size=batch_size,
                     validation_steps=2
                     )

In [ ]:
L2_KPI, L2_report, L2_matrix, L2_labels = ptk.evaluate_model(L2_model,test_data_generator)

In [ ]:
ptk.viz_confusion_matrix(L2_matrix,"Gradient L2 Kernal weight")

In [ ]:
ptk.viz_class_balance_comparison(L2_labels,"Gradient L2 Kernal weight")

In [ ]:
ptk.viz_training_history(L2_model_results,"Gradient L2 Kernal weight")

# Final Model

Now that the final model has been developed, it must be saved so that it can be used to make predictions on new data. To do this, we'll use the ModelCheckpoint callback provided by Keras.

The ModelCheckpoint callback allows us to save the model at specified intervals during training, based on a given metric. In this case, we want to save the model that has the best validation accuracy.

I will use a new ModelCheckpoint callback and add it to the list of callbacks, and pass to the fit method when we retrain the final model.

In [ ]:
# Building the model
final_model = Sequential()

# 3 convolutional layers
final_model.add(Conv2D(56, (4,4), activation='relu', input_shape = (224,224,1)))
final_model.add(MaxPooling2D(pool_size=(2,2)))

final_model.add(Conv2D(112, (2,2), activation='relu'))
final_model.add(MaxPooling2D(pool_size=(2,2)))

final_model.add(Conv2D(112, (2,2), activation='relu'))
final_model.add(MaxPooling2D(pool_size=(2,2)))

final_model.add(Flatten())

final_model.add(Dense(56,activation='relu',kernel_regularizer=l2(0.00005)))
final_model.add(Dense(56, activation='relu',kernel_regularizer=l2(0.0005)))

final_model.add(Dense(4))
final_model.add(Activation("softmax"))

# Compiling the final_model using some basic parameters
final_model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
final_callback = [
              ModelCheckpoint(filepath='data/final_model.h5',monitor='val_accuracy',save_best_only=True),
              EarlyStopping(monitor='val_accuracy', patience=5)
              ]

In [ ]:
final_model_results = final_model.fit(train_data_generator,
                                        callbacks=final_callback,
                                        batch_size=batch_size,
                                        steps_per_epoch=48, 
                                        epochs=50, 
                                        validation_data=test_data_generator, 
                                        validation_steps=2
                                        )

In [ ]:
final_KPI, final_report, final_matrix, final_labels = ptk.evaluate_model(final_model,test_data_generator)

In [ ]:
print(final_report)

In [ ]:
ptk.viz_confusion_matrix(final_matrix, "Final Model")

In [ ]:
ptk.viz_class_balance_comparison(final_labels,"Final Model")

In [ ]:
ptk.viz_training_history(final_model_results, "Final Model")

## Proof of Concept, Validation

In this section, I want to make sure that my saved final model can be loaded and utilized properly. To do this, I loaded the saved model and used the .evaluate() method on it. This allowed me to check the model's accuracy and loss on the testing data.

Next, I developed some code that invokes the model to make predictions on new data. I wanted to see how well the model can classify images that it has never seen before. The predictions were successful and accurate, which is a good sign that the model is working well.

In the [Coffee Classifier](coffee_classifier.py) python file you will find a PyQT5 application that provides a graphic user interface (GUI) with drag-and-drop functionality that can take an apropriately formatted image file as an input and utilizes the saved final model to make a prediction about the class of the input image. You can try it for yourself by first running and saving the final model in the previous section and then calling the Coffee Classifier app on the command line. 

Finally, I utilized the custom project_toolkit module to measure the performance of the model on the validation dataset. This is important because the validation dataset has not been used until this point of the project. By using this module here and throught this notebook, I am able to get the accuracy, loss, and other performance metrics as well as visualize them efficiently. This gave me a good idea of how well the model is performing on new data that it has not seen before.

In [ ]:
saved_model =  load_model('data/final_model.h5')

In [ ]:
saved_model.summary()

In [ ]:
results_train = saved_model.evaluate(train_data_generator)
print(f'Training Accuracy: {results_train[1]:.3}\nTraining Loss: {results_train[0]:.3}')

print('----------')

results_test = saved_model.evaluate(test_data_generator)
print(f'Test Accuracy: {results_test[1]:.3}\nTest Loss: {results_test[0]:.3}')


In [ ]:
def classify_bean(filepath:str):
    """
    Classifies the roast level of a coffee bean image.

    Args:
        filepath (str): The file path of the coffee bean image to classify.

    Returns:
         string: the predicted label as a string: "Green", "Light", "Medium" or "Dark".

    Raises:
        ValueError: If the input image is not a valid file or if the input image shape is not 224x224.
    """

    if not os.path.isfile(filepath):
        print("Error: {} is not a valid file path.".format(filepath))
        return

    else:
        # Load image and resize to model input shape
        img = load_img(filepath, target_size=(224, 224))

        # Convert image data to numpy array
        img_array = img_to_array(img)

        # Convert the image to grayscale
        img_array = np.dot(img_array, [0.2989, 0.5870, 0.1140])

        # Normalize the image array
        img_array /= 255.0

        # Expand dimensions to match the input shape of the model
        input_array = np.expand_dims(img_array, axis=0)
        input_array = np.expand_dims(input_array, axis=-1)

        # Make predictions
        predictions = saved_model.predict(input_array)
        predicted_label = np.argmax(predictions)

        label_dict = {0:"Green",1:"Light",2:"Medium",3:"Dark"}

        print("Predicted label:", label_dict[predicted_label])

        # return predicted_label

In [ ]:
classify_bean("data/test/C. Medium/medium (13).png")

## Validation Testing  
The validation sample used for testing the model was sourced from two different local coffee shops and one local roastery. The images were captured on a personal iPhone 13 using ambient fluorescent light and direct LED light set at 4500K with a white background, in order to replicate the lighting conditions of the kaggle dataset as much as possible.  

The model's performance on the validation sample was rather poor, with an overall accuracy of 38%. However, the patterns in the model's mistakes were consistent with my expectations. The model tended to classify the validation sample incorrectly towards the darker side, where raw coffee was sometimes classified as light, a light roast was classified as medium, and a medium roast was classified as dark, etc.

In [ ]:
val_dir = "data/val/"

val_datagen = ImageDataGenerator(rescale=1./255)

val_data_generator = val_datagen.flow_from_directory(
                       val_dir,
                       target_size=(224,224),
                       batch_size= batch_size,
                       class_mode='categorical',
                       color_mode='grayscale')

In [ ]:
val_KPI, val_report, val_matrix, val_labels = ptk.evaluate_model(saved_model,val_data_generator)

In [ ]:
ptk.viz_confusion_matrix(val_matrix,"Validation Sample")

In [ ]:
ptk.viz_class_balance_comparison(val_labels,"Validation Sample")

After consulting with the local roaster from whom I sourced the raw and light roast coffee beans for the validation set, I learned that the "light roast" in from the training and test samples would probably be considered "white coffee" by most roasters, which falls between raw and what he considers a "light roast." This information helped me understand why the model may have struggled with correctly classifying light roasts. It is probably more of a human labeling issue than a limitation of the models ability to distinguish between classes.  

Despite the poor performance on the validation sample, I am actually excited by the outcome. The model performed exceptionally well on the test data, which raises some suspicion as to whether the model was overfitting to the test dataset. Testing the model with truly unseen data was important, and the fact that the model underperformed in a way that was expected based on my conversation with the local roaster actually increased my confidence in the project's potential for a profitable production application.

# ![Project Scope and Limitations](images/readme/Scope%20and%20Limitations.png)

The scope of this project was to develop a convolutional neural network (CNN) model to classify images of coffee beans into four roast categories: green, light, medium, and dark. While this model can accurately classify individual bean images, it is important to note its limitations in a production setting.

One major limitation is that the model was trained on images of single beans, whereas in a production setting, the input data is likely to be a video feed of an entire roasting bin with thousands of coffee beans. It is unclear how the model will handle images of multiple beans that overlap with one another. Additionally, the model would need to be modified to take in video data or develop a data pipeline that preprocesses the video feed data into image data that can be input into the model (the latter is preferable).

Another limitation is that the entire training and test dataset came from the same roastery. Roasteries may differ in terms of what they consider light, medium, or dark roast. Some roasteries also sell blends, which mix multiple roasts together. Therefore, the model should be modified to output predictions on a gradient rather than four distinct classes to account for differences between what different roasteries consider light, medium, and dark and also account for blends that are somewhere in between these distinct classes.

Lastly, all of the training and test images were taken on the same iPhone 12 camera, and the validation dataset was captured on my personal iPhone 13 camera. Therefore, there may be inconsistencies in image quality and lighting conditions despite best efforts to recreate the resolution and lighting of the train/test dataset.

This project serves as a proof-of-concept that a CNN can be efficiently developed to address this classification problem. The project demonstrates that it is possible to take images from a variety of sources and preprocess them in a way that the model can take any of them as inputs. However, it is important to note that the model is far from being ready for use in production due to the aforementioned limitations.

# ![Recomendations and Conclusion](images/readme/Conclusion.png)
After developing and testing the model, I would like to share some recommendations and conclusions based on the project.

Firstly, I recommend expanding the dataset to include images of coffee beans from different roasteries, including images with multiple beans that overlap with one another. This will help improve the model's accuracy and ability to handle images from a variety of sources.

Secondly, I suggest beginning development of a video-to-photo data pipeline to preprocess video feed data into image data. This will require exploring the use of a pretrained model (probably in OpenCV) to perform object detection and generate bounding boxes, followed by taking snapshots of everything within each bounding box. With reliable bounding box generation, this approach should work with minimal issues throughout development.

Lastly, I recommend involving a software developer and UX designer in the project to focus on building an end-user application that can be easily installed and run on any machine. The application should be designed for non-technical users to easily utilize it with little to no specialized training.

In conclusion, this project is a successful proof-of-concept and worth investing resources into in order to bring it to production. Although there are limitations, this project has shown that it is possible to efficiently develop a CNN to address this classification problem and take images from a variety of sources, including within an application that non-technical users can interface with. If brought to production, this project has the potential to increase productivity and profitability by allowing for greater consistency in quality control and leveraging human capital by automating traditionally manual tasks. I am excited about this outcome and eagerly await any feedback.

## Contact Me:
I sincerely appreciate you taking the time to review this repository and hope you have gained some value from this data science project.  

If you have any questions, suggestions or otherwise in need of contacting me you can find me through:
- [LinkedIn](https://www.linkedin.com/in/zeth-abney/)
- [Instagram](https://www.instagram.com/texan_space_cowboy/)
- zethusabney@gmail.com

If you are curious about Flatiron School or data science bootcamps in general you can see my entire bootcamp journey on my youtube channel [Data Cowboy](https://www.youtube.com/channel/UCkYdKUId0iITN_czJ0X-sbA)

# Repository Stucture
├── [data](data) (png images of single coffe beans organized by class into subdirectories, gitignored)  
├── [images](images) (png and jpeg files used for embelishment and presentation)     
├── [presentation](presentation) (contains data visualizations, decorative graphics, pptx presentation files, pdf submission files, gitignored)  
├── [Coffee Classifier](notebook.ipynb) (source code for proof-of-concept GUI application)  
├── Notebook (you are here)   
├── [project_toolkit.py](project_toolkit.py) (source code for functions used for data visualization, model evaluation, and data preprocessing)  
└── [README](README.md) (In depth discussion of project discoveries)